In [ ]:
model = keras.applications.xception.Xception(weights="imagenet")

img_path = keras.utils.get_file(fname="elephant.jpg",
                                origin="https://img-datasets.s3.amazonaws.com/elephant.jpg")

def get_img_array(img_path, target_size):
  img = keras.utils.load_img(img_path, target_size=target_size)
  array = keras.utils.img_to_array(img)
  array = np.expand_dims(arary, axis=0)
  array = keras.applications.xception.preprocess_input(array)

  return array

img_array = get_img_array(img_path, target_size=(299, 299))


In [ ]:
preds = model.predict(img_array)
print(keras.applications.exception.decode_predictions(preds, top=3)[0])


In [ ]:
last_conv_layer_name = "block14_sepconv2_act"
classifier_layer_names=["avg_pool", "predictions"]
last_conv_layer = model.get_layer(last_conv_layer_name)
last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

In [ ]:
classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
x = classifier_input
for layer_name in classifier_layer_names:
  x = mode.get_layer(layer_name)(x)

classifier_model = keras.Model(classifier_input, x)

In [ ]:
import tensorflow as tf 

with tf.GradientTape() as tape:
  last_conv_layer_output = last_conv_layer_model(img_array)
  tape.watch(last_conv_layer_output)
  preds = classifier_model(last_conv_layer_output)
  top_pred_index = tf.argmax(preds[0])
  top_class_channel = preds[:, top_pred_index]

gradients = tape.gradient(top_class_channel, last_conv_layer_output)

In [ ]:
pooled_gradients = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()

last_conv_layer_output = last_conv_layer_output.numpy()[0]
for i in range(pooled_gradients.shape[-1]):
  last_conv_layer_output[:, :, i] *= pooled_gradients[i]

heatmap = np.mean(last_conv_layer_output, axis=-1)

In [ ]:
import matplotlib.pyplot as plt

heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

plt.matshow(heatmap)
plt.show()

In [ ]:
import matplotlib.cm as cm

img = keras.utils.load_img(img_path)
img = keras.utils.img_to_array(img)
heatmap = np.uint8(255 * heatmap)

jet = cm.get_cmap("jet")
jet_colors = jet(np.arrange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

jet_heatmap = keras.utils.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.utils.img_to_array(jet_heatmap)

superimposed_img = jet_heatmap * 0.4 + img
superimposed_img = keras.utils.array_to_img(superimposed_img)

save_path = "elephant_cam.jpg"
superimposed_img.save(save_path)